**Task 1**:

Stanford Manipulator

In [27]:
import numpy as np

def inverse_kinematics_stanford(H,d1,d2):
  
  R = [[H[0][0],H[0][1],H[0][2]],[H[1][0],H[1][1],H[1][2]],[H[2][0],H[2][1],H[2][2]]]
  [x_c,y_c,z_c] = [H[0][3],H[1][3],H[2][3]]

  theta_1 = np.arctan(y_c/x_c)

  r = (x_c**2+y_c**2)**0.5
  s = z_c - d1
  theta_2 = np.arctan(s/r) + np.pi/2

  d3 = (r**2 + s**2)**0.5

  DH = [[0,-np.pi/2,d1,theta_1],[0,np.pi/2,5,theta_2],[0,0,d3,0]]

  print('The DH parameters for the given configuration are:')
  return DH

H = [[0.6123,-0.5,0.6123,0.674],[0.353,0.866,0.353,7.317],[-0.5,0,0.707,4.243],[0,0,0,1]]
d1 = 5
d2 = 6
DH = inverse_kinematics_stanford(H,d1,d2)
print(DH)

n = len(DH)
def dhpara(n,DH,t=[0]*n):
  
  a = [0]*(n+1)
  alpha = [0]*(n+1)
  d = [0]*(n+1)
  theta = [0]*(n+1)
  for i in range(1,n+1):
    a[i] = DH[i-1][0]
    alpha[i] = DH[i-1][1]
    d[i] = DH[i-1][2]
    theta[i] = DH[i-1][3]
  
  A = [0]*(n+1)
  for i in range(1,n+1):
    A[i] = [[np.cos(theta[i]),-np.sin(theta[i])*np.cos(alpha[i]),np.sin(theta[i])*np.sin(alpha[i]),a[i]*np.cos(theta[i])],[np.sin(theta[i]),np.cos(theta[i])*np.cos(alpha[i]),-np.cos(theta[i])*np.sin(alpha[i]),a[i]*np.sin(theta[i])],[0,np.sin(alpha[i]),np.cos(alpha[i]),d[i]],[0,0,0,1]]

  T = [0]*(n+1)#intermediate transformation matrices

  #for the final transformation matrix, we multiply all the matrices
  T[1] = A[1]
  T_final = np.matmul(A[1],A[2])
  T[2] = T_final
  for i in range(3,n+1):
    T_final = np.matmul(T_final,A[i]) 
    T[i] = T_final

  #the final transformation matrix is stored in T_final
  pn = [0,0,d[n],1]
  po = np.matmul(T_final,pn)

  #end effector position:
  po = [po[0],po[1],po[2]]

  #to find manipulator jacobian

  J = [[0]*n]*6
  
  o = [0]*(n+1) #joint positions
  o[0] = [0,0,0]
  o[0] = np.array(o[0])
  for i in range(1,n+1):
    o[i] = [T[i][0][3],T[i][1][3],T[i][2][3]]
    o[i] = np.array(o[i])
  #print(o)
  
  z = [0]*(n+1) #coordinates of z axis
  z[0] = [0,0,1]
  z[0] = np.array(z[0])
  for i in range(1,n+1):
    z[i] = [T[i][0][2],T[i][1][2],T[i][2][2]]
    z[i] = np.array(z[i])
  #print(z)

  for i in range(1,n+1):#i defines column number and joint number
    if t[i-1]==0:#revolute joint
      Jv = np.cross(z[i],(o[n]-o[i]))
      J[0][i-1] = Jv[0]
      J[1][i-1] = Jv[1]
      J[2][i-1] = Jv[2]

      J[3][i-1] = z[i-1][0]
      J[4][i-1] = z[i-1][1]
      J[5][i-1] = z[i-1][2]

    if t[i-1]==1: #prismatic joint
      J[0][i-1] = z[i-1][0]
      J[1][i-1] = z[i-1][1]
      J[2][i-1] = z[i-1][2]

      J[3][i-1] = 0
      J[4][i-1] = 0
      J[5][i-1] = 0
  
  J = np.asmatrix(J)

  print('The final transformation matrix, end effector position and manipulator Jacobian for the configuration are:')
  return T_final, po, J

dhpara(n,DH,[0,0,1])


The DH parameters for the given configuration are:
[[0, -1.5707963267948966, 5, 1.4789412848546186], [0, 1.5707963267948966, 5, 1.4681369406336657], [0, 0, 7.386867671753705, 0]]
The final transformation matrix, end effector position and manipulator Jacobian for the configuration are:


(array([[ 9.39999608e-03, -9.95784291e-01,  9.12430045e-02,
         -4.30492146e+00],
        [ 1.02047138e-01,  9.17259276e-02,  9.90541638e-01,
          7.77562964e+00],
        [-9.94735151e-01,  5.49573011e-17,  1.02479161e-01,
          5.75700000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          1.00000000e+00]]),
 [-3.630921455016021, 15.092629637922753, 6.513999999999999],
 matrix([[1.000000e+00, 6.123234e-17, 0.000000e+00],
         [1.000000e+00, 6.123234e-17, 0.000000e+00],
         [1.000000e+00, 6.123234e-17, 0.000000e+00],
         [1.000000e+00, 6.123234e-17, 0.000000e+00],
         [1.000000e+00, 6.123234e-17, 0.000000e+00],
         [1.000000e+00, 6.123234e-17, 0.000000e+00]]))

**Task 2**:

SCARA Manipulator

In [28]:
def inverse_kin_scara(H,a1,a2):
  R = [[H[0][0],H[0][1],H[0][2]],[H[1][0],H[1][1],H[1][2]],[H[2][0],H[2][1],H[2][2]]]
  o = [H[0][3],H[1][3],H[2][3]]

  alpha = np.arctan(R[0][1]/R[0][0])
  c2 = (H[0][3]**2 + H[1][3]**2 - a1**2 - a2**2)/(2*a1*a2)

  theta_2 = np.arctan((1-c2)/c2)
  theta_1 = np.arctan(H[0][3]/H[1][3]) - np.arctan(a2*np.sin(theta_2)/(a1+a2*np.cos(theta_2)))
  d3 = o[2]

  DH = [[a1,0,0,theta_1],[a2,np.pi,0,theta_2],[0,0,d3,0]]
  
  print('The DH parameters for the given configuration are:')
  return DH

H = [[0.2588,0.9659,0,3.2849],[-0.9659,-0.2588,0,6.7955],[0,0,-1,-9],[0,0,0,1]]
DH = inverse_kin_scara(H,2,6)
print(DH)

n = 3
dhpara(n,DH,[0,0,1])


The DH parameters for the given configuration are:
[[2, 0, 0, 0.15473130081953068], [6, 3.141592653589793, 0, 0.3927826253079376], [0, 0, -9, 0]]
The final transformation matrix, end effector position and manipulator Jacobian for the configuration are:


(array([[ 8.53821325e-01,  5.20566177e-01,  6.37509702e-17,
          7.09903391e+00],
        [ 5.20566177e-01, -8.53821325e-01, -1.04562955e-16,
          3.43162629e+00],
        [ 0.00000000e+00,  1.22464680e-16, -1.00000000e+00,
          9.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          1.00000000e+00]]),
 [7.099033905294806, 3.4316262932091712, 18.0],
 matrix([[1., 1., 0.],
         [1., 1., 0.],
         [1., 1., 0.],
         [1., 1., 0.],
         [1., 1., 0.],
         [1., 1., 0.]]))

**Task 3**:

Finding joint velocities when Cartesian end effector velocities are given

In [29]:
import numpy as np

def inv_velocity(X_dot,J):
  #X_dot is cartesian velocity of end effector
  #J is manipulator Jacobian

  J_inv = np.linalg.pinv(J) #pseudo inverse of J
  q_dot = np.matmul(J_inv,X_dot)

  n = len(J[0])

  for i in range(n):
    print('The joint velocity q'+ str(i+1) +'_dot is '+ str(q_dot[i]))
  
  return q_dot

X_dot = [[12],[10],[5],[1],[4],[3]]
J = [[1,2,5],[2,3,6],[1,5,9],[1,5,2],[7,5,8],[1,0,2]]

inv_velocity(X_dot,J)

The joint velocity q1_dot is [-0.48521122]
The joint velocity q2_dot is [-0.69897343]
The joint velocity q3_dot is [1.55468176]


array([[-0.48521122],
       [-0.69897343],
       [ 1.55468176]])

**Task 6**:

Spherical Wrist

In [30]:
def spherical_wrist(H,d3):
  R = [[H[0][0],H[0][1],H[0][2]],[H[1][0],H[1][1],H[1][2]],[H[2][0],H[2][1],H[2][2]]]
  
  theta_1 = np.arctan(R[1][2]/R[0][2]) #pitch angle
  theta_2 = np.arctan(((1-R[2][2]**2)**0.5) / R[2][2]) #roll angle
  theta_3 = np.arctan(-R[2][1]/R[2][0]) #yaw angle

  DH = [[0,np.pi/2,0,theta_1],[0,np.pi/2,0,theta_2],[0,0,d3,theta_3]]

  print('The DH parameters for the given configuration are:')
  return DH

H = [[0.7146,-0.6131,0.3368,3.368],[0.6131,0.771,0.0594,0.594],[-0.296,0.171,0.939,9.39],[0,0,0,1]]
spherical_wrist(H,10)

The DH parameters for the given configuration are:


[[0, 1.5707963267948966, 0, 0.17457056954959707],
 [0, 1.5707963267948966, 0, 0.3510853369209009],
 [0, 0, 10, 0.5238630603949287]]

**Task 7**:

PPP Configuration

In [31]:
DH = [[0,-np.pi/2,5,0],[0,-np.pi/2,10,-np.pi/2],[0,0,8,0]]
n=3
t = [1,1,1]

dhpara(n,DH,t)

The final transformation matrix, end effector position and manipulator Jacobian for the configuration are:


(array([[ 6.123234e-17,  6.123234e-17,  1.000000e+00,  8.000000e+00],
        [-6.123234e-17, -1.000000e+00,  6.123234e-17,  1.000000e+01],
        [ 1.000000e+00, -6.123234e-17, -6.123234e-17,  5.000000e+00],
        [ 0.000000e+00,  0.000000e+00,  0.000000e+00,  1.000000e+00]]),
 [16.0, 10.0, 4.999999999999999],
 matrix([[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]]))

**Task 8**:

Inverse Kinematics for PPP Configuration

In [32]:
import numpy as np

def inv_kin_ppp(H):
  d1 = H[0][3]
  d2 = H[1][3]
  d3 = H[2][3]

  DH = [[0,-np.pi/2,d1,0],[0,-np.pi/2,d2,-np.pi/2],[0,0,d3,0]]

  print('The DH parameters for the given configuration are:')
  return DH

H = [[0,0,1,3],[0,1,0,8],[1,0,0,6],[0,0,0,1]]
inv_kin_ppp(H)

The DH parameters for the given configuration are:


[[0, -1.5707963267948966, 3, 0],
 [0, -1.5707963267948966, 8, -1.5707963267948966],
 [0, 0, 6, 0]]